* contributions of DQN
> 1. understand image itself as states(input)
  2. can make better performance than human
  3. with replay memory, solve the problem of the relation b/t samples

* limits of DQN
> 1. requires huge memory
  2. low training speed
  3. unstable policy training b/c it is value based


## A3C (Asynchronous Advantage Actor-Critic)
1. solve the problem of the relation b/t samples with asynchronous updates
2. does not use replay memory
3. can apply policy gradient algorithm (actor-critic)
4. relatively high training speed b/c it uses many agents in each envir.<br />


>A3C = Asynchronous update + Actor-Critic
>> Actor-Critic = REINFORCE + real-time training
>>> REINFORCE = Monte Carlo policy gradient


### Policy gradient

###### cf) In supervised learning,
<img src="./figs/supervise1.png">

> $\sum y_i logp_i$ represents the difference b/t predcition(p) and answer(y)
and then, gradient descent update the network

##### In reinforcement learning, 
<img src="./figs/pg1.png">

> $\sum y_i logp_i$ represents the difference b/t predcition(p) and answer(y)

<img src="./figs/pg2.png">

> Need to think the meaning of making the predcition(p) to the action taken(y)

Calcuating $Q(s_t, a_t)$, how good the action($a_t$) taken on the current state($s_t$) is, the process of the above teach the agent how to act.<br /> 
($Q_\pi(s, a) = E_\pi[R{t+1} + \gamma R{t+2} + ... | s_t=s, a_t=a]$) 
<img src="./figs/pg3.png"><br />
Specifically, the network is trained to predict the action which makes the agent to do the action providing better rewards. By training according to the gradient descent of the difference b/t prediction(p) and answer(y) multiplied by Q value function, the agent will be aware of which action is  good for better rewards. In other words, policy gradient in RL increase the probability of the action providing better rewards.

To sum up, within policy gradient, there are two important factors:
* the standard to update the policy: objective function
* the method to update the policy with the objective function: gradient ascent
> Policy: $\pi_\theta (a|s) = P[a_t=a|s_t=s, \theta]$

#### Objective func. , $J(\theta)$
: produce the information about which direction for the policy to be updated     

> $ J(\theta) = E[\sum^{T-1}_{t=0} r_{t+1}|\pi_{\theta}]$ 
              = $E[r_1 + r_2 + r_3 + ... + r_T|\pi_\theta]$
>>  cumulated expected rewards according to a trajectory


With the above objective func., <br>
the $\theta$ can be updated by the bellow: <br>

<br>
<div align="center">$\theta' = \theta + \alpha \nabla_\theta J(\theta)$</div>
<div align="center">($\nabla_\theta J(\theta)$: policy gradient)</div>

#### In REINFORCE algorithm,<br>
<div align="center">$\nabla_\theta J(\theta) ~ \sum^{T-1}_{t=0} \nabla_\theta log\pi_\theta(a_t|s_t)G_t$</div><br>


$J(\theta) = E[\sum^{T-1}_{t=0}r_{t+1}|\pi_\theta]$<br><br>
$ \ \ \ \ \ \ \  = E[r_1|\pi_\theta] + E[r_2|\pi_\theta] + ... + E[r_T|\pi_\theta]$<br><br>
According to the fact that expected value is probability of ($x$) * output of ($x$),

<img src="./figs/REINFORCE1.png"> <br>
Differenciating the both sides, 
<img src="./figs/REINFORCE2.png" width="400"><br>
The final step can be done by the bellow:
<img src="./figs/REINFORCE3.png" width="500"><br>

To sum up, 
<img src="./figs/REINFORCE4.png" width="400">

Then, the total sum of any probability is 1 and log(1) = 0, 
<img src="./figs/REINFORCE5.png" width="500">
<img src="./figs/REINFORCE6.png" width="500"><br>
Differenciating the both sides, 
<img src="./figs/REINFORCE7.png" width="500">
<br>
<br>

_________________
Finally, 
<img src="./figs/REINFORCE8.png" > <br>

With a discount factor,
<img src="./figs/REINFORCE9.png" > <br>
<img src="./figs/REINFORCE10.png" > <br>


#### Pseudo code for REINFORCE algorithm
1. execute the policy within one episode and record trajectory
2. according to one episode/trajectory, calculate $G_t$
3. update policy based on the policy gradient <br>
$ \ \ \  ( policy gradient = \sum^{T-1}_{t=0}\nabla_\theta log\pi_\theta(a_t|s_t)G_t )$
4. go to 1
<br>
<br>

> update episode by episode = MCT policy gradient = REINFORCE


<U>However, there are some limits of REINFORCE</U>:
1. update episode by spisode (not online)
2. high variance

#### Actor-Critic method with advantage function

To overcome those limits,
1. Apply actor-critic method on REINFORCE

<img src="./figs/AC1.png" width="500">
<img src="./figs/AC2.png" width="500"><br>
If we know $Q_{\pi_{\theta}} (s_t, a_t)$, we can update the net. step by step. Thus, need to approximate it:<br><br>
<div align="center"> $Q_{\pi_{\theta}} (s_t, a_t) \ $~$ \ Q_w(s_t, a_t)$</div>

> Critic will do this!!!    
>> Accordingly, <br><br>
>> $\nabla_\theta J(\theta) \ \ $~$ \ \ \nabla_\theta log\pi_\theta (a_t|s_t)Q_w(s_t, a_t)$


2. apply advantage function(Q value function - baseline value function)
<br>
<div align="center">$\nabla_\theta J(\theta) \ \ $~$ \ \ \nabla_\theta log\pi_\theta (a_t|s_t)(Q_w(s_t, a_t) - V_v(s_t))$


In this process, it costs too much to approximate both of the Q value function and the value function. Thus, we can use 
<div align="center">$E[r_{t+1} + \gamma V(s_{t+1}) | s_t, a_t]$</div>
<br>
Then, <br>
<div align="center">$\nabla_\theta J(\theta) \ \ $~$ \ \ \nabla_\theta log\pi_\theta (a_t|s_t)(r_{t+1} + \gamma V_v(s_{t+1}) - V_v(s_t))$</div>

_________________
> Actor: 
>>        1) approximate $\theta$
>>        2) update with 
<div align="center">$\nabla_\theta J(\theta) \ \ $~$ \ \ \nabla_\theta 
log\pi_\theta (a_t|s_t)(r_{t+1} + \gamma V_v(s_{t+1}) - V_v(s_t))$</div>


> Critic: 
>>        1) approximate value function
>>        2) update with the error,
<div align="center">$(r_{t+1} + \gamma V_v(s_{t+1}) - V_v(s_t))^2$</div>

<br>
<img src="./figs/AC3.png">
<br>
<img src="./figs/AC4.png">

#### In Asynchronous update, 
Each agent has its own independent environment and they are called local networks.
<img src="./figs/a3c1.png"><br >
then, the global network is updated by and updates those each local networks each other <U>asynchronously</U>.  
_____________
1. Multi-step loss function <br>
<img src="./figs/a3c2.png"><br >
<img src="./figs/a3c3.png"><br >
<br><br>

2. Entropy loss function <br>
<img src="./figs/a3c4.png"><br >
<img src="./figs/a3c5.png"><br >
<img src="./figs/a3c6.png"><br >
